In [1]:
import pandas as pd
import json
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder 
import pickle
import xgboost
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
print(xgboost.__version__) 

0.90


In [3]:
business_features = pd.read_csv('/data/chaoyu/data553/business_train.csv')
user_features = pd.read_csv('/data/chaoyu/data553/user_train.csv')
embeddings = pd.read_csv('/data/chaoyu/data553/emb_city.csv')

In [4]:
df_business = pd.read_json('/data/chaoyu/data553/business.json', lines=True)
df_user = pd.read_json('/data/chaoyu/data553/user.json', lines=True)

In [5]:
df_train = pd.read_csv('/data/chaoyu/data553/yelp_train.csv')

In [6]:
df_val = pd.read_csv('/data/chaoyu/data553/yelp_val.csv')
df_val

,user_id,business_id,stars
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg,5.0
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ,5.0
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA,4.0
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA,5.0
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw,3.0
...,...,...,...
142039,pA9NXgASl86RImkdBtydrA,q6-SF8zHFU1AWO70k92o1Q,2.0
142040,_eUb7UGsUoSfi9n2ieF5ow,hgWMxKhrnOUd3m5nOUBIkA,4.0
142041,cEJGXB63KhROA-XmE_jgXw,0ldxjei8v4q95fApIei3Lg,5.0
142042,Z4-V0hc51oxUdULWJOufeg,j29tuUdrfaxmGjwxHdHZPA,3.0


In [7]:
try:
    df_val.drop(['stars'], axis=1, inplace=True)
except:
    pass
df_val

,user_id,business_id
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw
...,...,...
142039,pA9NXgASl86RImkdBtydrA,q6-SF8zHFU1AWO70k92o1Q
142040,_eUb7UGsUoSfi9n2ieF5ow,hgWMxKhrnOUd3m5nOUBIkA
142041,cEJGXB63KhROA-XmE_jgXw,0ldxjei8v4q95fApIei3Lg
142042,Z4-V0hc51oxUdULWJOufeg,j29tuUdrfaxmGjwxHdHZPA


In [8]:
val_user = set(df_val['user_id'].tolist())
len(val_user)

11270

In [9]:
val_business = set(df_val['business_id'].tolist())
len(val_business)

22120

In [10]:
business_list = set(df_business['business_id'].tolist())
#business_list = set(df_train['business_id'].tolist())
len(business_list)

188593

In [11]:
user_list = set(df_user['user_id'].tolist())
#user_list = set(df_train['user_id'].tolist()[:len(df_train['user_id'].tolist())//10])
len(user_list)

1518169

In [35]:
comb = []
users = []
businesses = []
nulls = []
user_null = []
business_null = []
null = []
comb_list = embeddings['id'].tolist()
#comb_list = []
for i in range(len(df_val)):
    business_id = df_val['business_id'][i]
    user_id = df_val['user_id'][i]
    if (user_id in comb_list) and (business_id in comb_list):
        comb.append(i)
    elif (user_id in comb_list) and (business_id not in comb_list) and (business_id in business_list):
        comb.append(i)
    elif (user_id not in comb_list) and (business_id in comb_list) and (user_id in user_list):
        businesses.append(i)
    elif (user_id not in comb_list) and (business_id not in comb_list) and (user_id in user_list) and (business_id in business_list):
        nulls.append(i)
    elif (user_id not in comb_list) and (business_id not in comb_list) and (user_id not in user_list) and (business_id in business_list):
        business_null.append(i)
    elif (user_id not in comb_list) and (business_id not in comb_list) and (user_id in user_list) and (business_id not in business_list):
        user_null.append(i)
    else:
        null.append(i)

df_comb = pd.DataFrame(df_val.iloc[comb])
df_users = pd.DataFrame(df_val.iloc[users])
df_businesses = pd.DataFrame(df_val.iloc[businesses])
df_nulls = pd.DataFrame(df_val.iloc[nulls])
df_user_null = pd.DataFrame(df_val.iloc[user_null])
df_business_null = pd.DataFrame(df_val.iloc[business_null])
df_null = pd.DataFrame(df_val.iloc[null])

In [36]:
df_comb

,user_id,business_id
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw
...,...,...
142039,pA9NXgASl86RImkdBtydrA,q6-SF8zHFU1AWO70k92o1Q
142040,_eUb7UGsUoSfi9n2ieF5ow,hgWMxKhrnOUd3m5nOUBIkA
142041,cEJGXB63KhROA-XmE_jgXw,0ldxjei8v4q95fApIei3Lg
142042,Z4-V0hc51oxUdULWJOufeg,j29tuUdrfaxmGjwxHdHZPA


In [37]:
df_users

,user_id,business_id


In [38]:
df_businesses

,user_id,business_id


In [39]:
df_nulls

,user_id,business_id


In [40]:
df_user_null

,user_id,business_id


In [41]:
df_business_null

,user_id,business_id


In [42]:
df_null

,user_id,business_id


In [43]:
comb_data = df_comb.merge(embeddings, left_on='business_id', right_on='id', how='left').set_axis(df_comb.index)
comb_data = comb_data.merge(embeddings, left_on='user_id', right_on='id', how='left', suffixes=('_bus', '_user')).set_axis(comb_data.index)
comb_data = comb_data.merge(business_features, on='business_id', how='left').set_axis(comb_data.index)
comb_data = comb_data.merge(user_features, on='user_id', how='left').set_axis(comb_data.index)
comb_data.drop(['user_id', 'business_id', 'id_bus', 'id_user'], axis=1, inplace=True)
comb_data

,emb_0_bus,emb_1_bus,emb_2_bus,emb_3_bus,emb_4_bus,emb_5_bus,emb_6_bus,emb_7_bus,emb_8_bus,emb_9_bus,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u
0,0.583845,-0.651579,0.772014,0.743067,0.031108,0.148580,0.473354,0.233632,-0.542401,0.183733,...,0,0,0,2,4,0,0,1,0,118
1,0.464730,-0.374081,0.801788,-0.679268,-0.771335,0.144709,0.649751,-0.857616,-1.330940,0.666551,...,1,0,0,0,3,8,8,4,2,141
2,-0.245982,-0.296467,2.582823,0.197992,1.301252,0.167199,-0.773905,-0.046012,-0.107178,-0.615027,...,2,0,0,15,11,10,10,13,10,251
3,-0.571106,-1.108008,-0.359693,0.905840,1.058916,-0.917500,-0.120490,0.367527,-0.243312,-0.415801,...,0,1,1,16,23,47,47,13,0,104
4,0.067405,-0.006672,0.993188,0.376995,-0.771937,-0.489112,-0.068178,0.508360,-0.111899,0.735718,...,3,0,0,8,11,11,11,15,5,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142039,0.768285,0.438905,0.596470,0.559148,0.184115,0.382647,-0.567784,-0.833221,0.204782,0.146390,...,0,1,1,1,3,0,0,2,2,56
142040,0.413280,0.211834,0.828025,0.080972,-0.013818,-1.843138,-0.546528,-0.162567,0.681033,0.655531,...,0,0,0,1,1,0,0,0,0,59
142041,0.778928,-0.863108,1.741980,0.007115,0.273352,-0.786310,-1.423493,0.813603,0.535018,1.395781,...,0,0,0,1,2,1,1,1,1,123
142042,0.654608,-1.037812,-0.363996,0.045705,-0.235668,-0.459663,-0.557290,-0.231956,-0.689421,-0.586433,...,0,0,0,0,0,0,0,0,0,55


In [44]:
dense = ['stars', 'review_count_b', 'review_count_u', 'useful', 'funny', 'cool', 'fans', 
               'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 
               'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 
               'compliment_writer', 'compliment_photos']
if len(comb_data) > 0:
    mms = MinMaxScaler(feature_range=(0,1))
    comb_data[dense] = mms.fit_transform(comb_data[dense])
comb_data

,emb_0_bus,emb_1_bus,emb_2_bus,emb_3_bus,emb_4_bus,emb_5_bus,emb_6_bus,emb_7_bus,emb_8_bus,emb_9_bus,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u
0,0.583845,-0.651579,0.772014,0.743067,0.031108,0.148580,0.473354,0.233632,-0.542401,0.183733,...,0.000000,0.000000,0.000000,0.000275,0.000252,0.000000,0.000000,0.000125,0.000000,0.007304
1,0.464730,-0.374081,0.801788,-0.679268,-0.771335,0.144709,0.649751,-0.857616,-1.330940,0.666551,...,0.000364,0.000000,0.000000,0.000000,0.000189,0.000429,0.000429,0.000502,0.000053,0.009235
2,-0.245982,-0.296467,2.582823,0.197992,1.301252,0.167199,-0.773905,-0.046012,-0.107178,-0.615027,...,0.000728,0.000000,0.000000,0.002060,0.000692,0.000536,0.000536,0.001631,0.000267,0.018470
3,-0.571106,-1.108008,-0.359693,0.905840,1.058916,-0.917500,-0.120490,0.367527,-0.243312,-0.415801,...,0.000000,0.000444,0.000766,0.002198,0.001447,0.002518,0.002518,0.001631,0.000000,0.006129
4,0.067405,-0.006672,0.993188,0.376995,-0.771937,-0.489112,-0.068178,0.508360,-0.111899,0.735718,...,0.001092,0.000000,0.000000,0.001099,0.000692,0.000589,0.000589,0.001882,0.000134,0.018890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142039,0.768285,0.438905,0.596470,0.559148,0.184115,0.382647,-0.567784,-0.833221,0.204782,0.146390,...,0.000000,0.000444,0.000766,0.000137,0.000189,0.000000,0.000000,0.000251,0.000053,0.002099
142040,0.413280,0.211834,0.828025,0.080972,-0.013818,-1.843138,-0.546528,-0.162567,0.681033,0.655531,...,0.000000,0.000000,0.000000,0.000137,0.000063,0.000000,0.000000,0.000000,0.000000,0.002351
142041,0.778928,-0.863108,1.741980,0.007115,0.273352,-0.786310,-1.423493,0.813603,0.535018,1.395781,...,0.000000,0.000000,0.000000,0.000137,0.000126,0.000054,0.000054,0.000125,0.000027,0.007724
142042,0.654608,-1.037812,-0.363996,0.045705,-0.235668,-0.459663,-0.557290,-0.231956,-0.689421,-0.586433,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002015


In [45]:
users_data = df_users.merge(embeddings, left_on='user_id', right_on='id', how='left').set_axis(df_users.index)
users_data = users_data.merge(business_features, on='business_id', how='left').set_axis(users_data.index)
users_data = users_data.merge(user_features, on='user_id', how='left').set_axis(users_data.index)
users_data.drop(['user_id', 'business_id', 'id'], axis=1, inplace=True)
users_data

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u


In [46]:
if len(users_data) > 0:
    mms = MinMaxScaler(feature_range=(0,1))
    users_data[dense] = mms.fit_transform(users_data[dense])
users_data

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u


In [47]:
businesses_data = df_businesses.merge(embeddings, left_on='business_id', right_on='id', how='left').set_axis(df_businesses.index)
businesses_data = businesses_data.merge(business_features, on='business_id', how='left').set_axis(businesses_data.index)
businesses_data = businesses_data.merge(user_features, on='user_id', how='left').set_axis(businesses_data.index)
businesses_data.drop(['user_id', 'business_id', 'id'], axis=1, inplace=True)
businesses_data

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u


In [48]:
if len(businesses_data) > 0:
    mms = MinMaxScaler(feature_range=(0,1))
    businesses_data[dense] = mms.fit_transform(businesses_data[dense])
businesses_data

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u


In [49]:
nulls_data = df_nulls.merge(business_features, on='business_id', how='left').set_axis(df_nulls.index)
nulls_data = nulls_data.merge(user_features, on='user_id', how='left').set_axis(nulls_data.index)
nulls_data.drop(['user_id', 'business_id'], axis=1, inplace=True)
nulls_data

,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u


In [50]:
if len(nulls_data) > 0:
    mms = MinMaxScaler(feature_range=(0,1))
    nulls_data[dense] = mms.fit_transform(nulls_data[dense])

In [51]:
temp_user_list = []
for i in df_user_null['user_id'].keys():
    temp_user_list.append(df_user[df_user['user_id'] == df_user_null['user_id'][i]]['average_stars'].values[0])
df_user_null.insert(df_user_null.shape[1], 'pred', temp_user_list)
df_user_null

,user_id,business_id,pred


In [52]:
temp_business_list = []
for i in df_business_null['business_id'].keys():
    temp_business_list.append(df_business[df_business['business_id'] == df_business_null['business_id'][i]]['stars'].values[0])
df_business_null.insert(df_business_null.shape[1], 'pred', temp_business_list)
df_business_null

,user_id,business_id,pred


In [53]:
df_null.insert(df_null.shape[1], 'pred', 3.75)
df_null

,user_id,business_id,pred


In [ ]:
comb_res = []
for i in range(10):
    model = pickle.load(open("/home/chaoyu/553models/HIN2Vec_mms/model_" + str(i) + "_new.pkl", 'rb'))
    comb_res.append(list(model.predict(comb_data)))
    del model
comb_res = np.array(comb_res)
y_pred = np.mean(comb_res, axis=0)
df_comb.insert(df_comb.shape[1], 'pred', y_pred)
df_comb

[00:19:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
users_res = []
for i in range(10):
    model = pickle.load(open("/home/chaoyu/553models/HIN2Vec_mms_user/model_" + str(i) + "_user.pkl", 'rb'))
    users_res.append(list(model.predict(users_data)))
    del model
users_res = np.array(users_res)
y_pred = np.mean(users_res, axis=0)
df_users.insert(df_users.shape[1], 'pred', y_pred)
df_users

In [ ]:
businesses_res = []
for i in range(10):
    model = pickle.load(open("/home/chaoyu/553models/HIN2Vec_mms_business/model_" + str(i) + "_business.pkl", 'rb'))
    businesses_res.append(list(model.predict(businesses_data)))
businesses_res = np.array(businesses_res)
y_pred = np.mean(businesses_res, axis=0)
df_businesses.insert(df_businesses.shape[1], 'pred', y_pred)
df_businesses

In [ ]:
nulls_res = []
for i in range(10):
    model = pickle.load(open("/home/chaoyu/553models/HIN2Vec_mms_null/model_" + str(i) + "_null.pkl", 'rb'))
    nulls_res.append(list(model.predict(nulls_data)))
nulls_res = np.array(nulls_res)
y_pred = np.mean(nulls_res, axis=0)
df_nulls.insert(df_nulls.shape[1], 'pred', y_pred)
df_nulls

In [ ]:
combination = pd.concat([df_comb, df_businesses, df_users, df_nulls, df_business_null, df_user_null, df_null]).sort_index()
combination

In [ ]:
df_test = pd.read_csv('/data/chaoyu/data553/yelp_val.csv')
df_test

In [ ]:
mse = mean_squared_error(combination['pred'], df_test['stars'])
print("Val RMSE: %f" % (mse ** 0.5))

In [ ]:
df_user_new = df_user[['user_id', 'average_stars']]
df_user_new

In [ ]:
df_user_new.to_csv('user_name.csv', index=False)

In [ ]:
df_business_new = df_business[['business_id', 'stars']]
df_business_new

In [ ]:
df_business_new.to_csv('business_name.csv', index=False)